https://deepmlblog.wordpress.com/2016/01/05/residual-networks-in-torch-mnist/

In [40]:
require 'torch'
require 'nn'
require 'optim'
require 'image'
torch.manualSeed(1)

function reload(name)
    package.loaded[name] = nil
    return require(name)

end
utils = reload('./utils')
models = reload('./models')
Logger = reload('./logger')


torch.manualSeed(1)

In [2]:

train_data, train_label,
    validation_data, validation_label = utils.load_data()

train_data = train_data[{{1,1000}}]
train_label = train_label[{{1, 1000}}]


In [42]:

SEED = 1
TITLE = 'mnist_mlp'
LOGPATH = './results/' .. TITLE
EPOCHS = 100

torch.manualSeed(SEED)

model = models.mlp()
criterion = nn.ClassNLLCriterion()

sgd_params = {
   learningRate = 1e-2,
   learningRateDecay = 0,
   weightDecay = 0,
   momentum = 0
}

x, dl_params = model:getParameters()

info_log = Logger(
    LOGPATH, 
    {
        general_info = {
            title = TITLE,
            description = '',
            seed = SEED,
            model = tostring(model),
            criterion = tostring(criterion),
        },
        data_size = {
            train_data = torch.totable(train_data:size()),
            train_label = torch.totable(train_label:size()),
            validation_data = torch.totable(validation_data:size()),
            validation_label = torch.totable(validation_label:size()),
        },
        sgd_params = sgd_params
    })
info_log:write()

logger = optim.Logger('./results/mnist_mlp/accuracy.log')
logger:setNames{'duration', 'train_loss', 'validation_loss', 'learning_rate'}

time_start = os.time()
for i=1,EPOCHS do
    
    batch_data = train_data
    batch_label = train_label
    
    sys.tic()
    _, fs = optim.sgd(utils.feval,x,sgd_params)
    duration = sys.toc()
    local train_loss = fs[1]
    
    logger:add{duration, train_loss, validation_loss, sgd_params.learningRate}
    
    validation_loss = calc_validation_loss(model, criterion, validation_data, validation_label)

end

total_train_time = os.time() - time_start
info_log:write_time(total_train_time)

logger:style{'+-', '+-', '+-', '+-', '+-'}
logger:plot()
